<a href="https://colab.research.google.com/github/Harsh-Sarvaiya/AI-Assisted-System-Design-Paper/blob/main/Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plantuml
!pip install python-docx
!pip install openai==0.28

In [ ]:
import openai
import pandas as pd
from docx import Document
import time
import os

# Optional: Install PlantUML if you plan to generate images
# !pip install plantuml

from plantuml import PlantUML

# Load user stories from CSV
df = pd.read_csv("/content/drive/MyDrive/Research with Professor - GPT to SD/Papers and data/Selected_User_Stories.csv")
user_stories = df["Random Line"].tolist()

# Define OpenAI API key securely
openai.api_key = "API_KEY"  # Recommended to set as an environment variable

# Function to get use case scenarios in UML format with additional details
def get_use_case_scenarios_in_uml(user_story):
    """
    Uses OpenAI API to generate detailed use case scenarios in UML format for a user story.
    Includes main flow, alternate flow, triggers, actors, preconditions, and postconditions.
    """
    prompt = f"""
    You are an expert in UML use case design. For the following user story, generate a complete UML use case scenario with detailed sections:
    - Use Case Name: A short descriptive name for the use case.
    - Actors: List all actors involved.
    - Preconditions: Any conditions that must be met before the use case starts.
    - Triggers: The events or conditions that start the use case.
    - Main Flow of Events: A step-by-step description of the primary flow.
    - Alternate Flow of Events: Any alternate or exception flows.
    - Postconditions: The final state after the use case completes.

    User Story: "{user_story}"

    Ensure the output is complete and does not contain placeholders such as "### Use Case Name".
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert in UML use case design and software engineering."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2000,
            temperature=0.3
        )
        response_text = response['choices'][0]['message']['content'].strip()
        if "### Use Case Name" in response_text or not response_text:
            response_text = "Error: Unable to generate detailed use case scenario."
        return response_text
    except Exception as e:
        print(f"Error for story: {user_story} - {e}")
        return "Error: Unable to generate use case scenario"

# Function to convert use case scenario to PlantUML sequence diagram script
def get_plantuml_sequence_diagram(use_case_scenario):
    """
    Uses OpenAI API to generate a PlantUML sequence diagram script based on the use case scenario.
    """
    prompt = f"""
    You are an expert in software engineering and UML diagramming. Based on the following UML use case scenario, generate a PlantUML script for a sequence diagram that visualizes the main flow of events.

    Use Case Scenario:
    {use_case_scenario}

    The PlantUML script should include all relevant actors and system components involved in the main flow.
    Ensure the script is syntactically correct and can be rendered without errors.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert in software engineering and UML diagramming."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0.3
        )
        plantuml_script = response['choices'][0]['message']['content'].strip()
        if "Error" in plantuml_script or not plantuml_script:
            plantuml_script = "@startuml\n' Error: Unable to generate sequence diagram\n@enduml"
        return plantuml_script
    except Exception as e:
        print(f"Error generating PlantUML script - {e}")
        return "@startuml\n' Error: Unable to generate sequence diagram\n@enduml"

# Directory to save PlantUML scripts and diagrams
plantuml_dir = "plantuml_sequences"
os.makedirs(plantuml_dir, exist_ok=True)

# Initialize PlantUML server (optional, if you have a local PlantUML server)
# If you don't have a local server, you can use the default PlantUML public server
plantuml_server = PlantUML(url='http://www.plantuml.com/plantuml/img/')  # Public server

# Generate use case scenarios and corresponding PlantUML sequence diagrams
use_case_results = []
plantuml_scripts = []
sequence_diagrams = []

for idx, story in enumerate(user_stories, start=1):
    print(f"Processing User Story {idx}/{len(user_stories)}")
    use_case = get_use_case_scenarios_in_uml(story)
    use_case_results.append((story, use_case))

    # Generate PlantUML script
    plantuml_script = get_plantuml_sequence_diagram(use_case)
    plantuml_scripts.append((story, plantuml_script))

    # Save PlantUML script to a file
    sanitized_story = "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in story])[:50]
    puml_filename = f"{plantuml_dir}/sequence_{idx}_{sanitized_story}.puml"
    with open(puml_filename, 'w') as f:
        f.write(plantuml_script)

    # Optionally, generate and save the sequence diagram image
    try:
        # Using PlantUML public server to generate the image
        diagram_url = plantuml_server.get_url(plantuml_script)
        diagram_filename = f"{plantuml_dir}/sequence_{idx}_{sanitized_story}.png"
        # Download the image
        import requests
        img_data = requests.get(diagram_url).content
        with open(diagram_filename, 'wb') as img_file:
            img_file.write(img_data)
    except Exception as e:
        print(f"Error generating diagram for story {idx}: {e}")

    time.sleep(1)  # Add delay to avoid rate limits

# Save use case scenarios to a Word document
doc = Document()
doc.add_heading('UML Use Case Scenarios', level=1)

for story, use_case in use_case_results:
    doc.add_heading(f"User Story: {story}", level=2)
    doc.add_paragraph(use_case)
    doc.add_paragraph("\n")  # Add a blank line between entries

doc.save("use_case_scenarios.docx")

# Save PlantUML scripts to a separate Word document (optional)
puml_doc = Document()
puml_doc.add_heading('PlantUML Sequence Diagrams', level=1)

for story, script in plantuml_scripts:
    puml_doc.add_heading(f"User Story: {story}", level=2)
    puml_doc.add_paragraph(script)
    puml_doc.add_paragraph("\n")  # Add a blank line between entries

puml_doc.save("plantuml_sequence_diagrams.docx")

# Notify the user
print("Word documents saved as 'use_case_scenarios.docx' and 'plantuml_sequence_diagrams.docx'.")
print(f"PlantUML scripts and sequence diagram images are saved in the '{plantuml_dir}' directory.")

# Download the Word files in Google Colab (if applicable)
try:
    from google.colab import files
    files.download('use_case_scenarios.docx')
    files.download('plantuml_sequence_diagrams.docx')
    # Optionally, zip the PlantUML directory and download
    import shutil
    shutil.make_archive("plantuml_sequences", 'zip', plantuml_dir)
    files.download("plantuml_sequences.zip")
except ImportError:
    print("Google Colab is not being used, files saved locally.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import openai
import pandas as pd
from docx import Document
import time
import os

# Optional: Install PlantUML if you plan to generate images
# !pip install plantuml

from plantuml import PlantUML

# Load user stories from CSV
df = pd.read_csv("/content/drive/MyDrive/Research with Professor - GPT to SD/Papers and data/Selected_User_Stories.csv")
user_stories = df["Random Line"].tolist()

# Define OpenAI API key securely
openai.api_key = "API-KEY"  # Recommended to set as an environment variable

# # Function to get use case scenarios in UML format with additional details
# def get_use_case_scenarios_in_uml(user_story):
#     """
#     Uses OpenAI API to generate detailed use case scenarios in UML format for a user story.
#     Includes main flow, alternate flow, triggers, actors, preconditions, and postconditions.
#     """
#     prompt = f"""
#     You are an expert in UML use case design. For the following user story, generate a complete UML use case scenario with detailed sections:
#     - Use Case Name: A short descriptive name for the use case.
#     - Actors: List all actors involved.
#     - Preconditions: Any conditions that must be met before the use case starts.
#     - Triggers: The events or conditions that start the use case.
#     - Main Flow of Events: A step-by-step description of the primary flow.
#     - Alternate Flow of Events: Any alternate or exception flows.
#     - Postconditions: The final state after the use case completes.

#     User Story: "{user_story}"

#     Ensure the output is complete and does not contain placeholders such as "### Use Case Name".
#     """
#     try:
#         response = openai.ChatCompletion.create(
#             model="gpt-4",
#             messages=[
#                 {"role": "system", "content": "You are an expert in UML use case design and software engineering."},
#                 {"role": "user", "content": prompt}
#             ],
#             max_tokens=2000,
#             temperature=0.3
#         )
#         response_text = response['choices'][0]['message']['content'].strip()
#         if "### Use Case Name" in response_text or not response_text:
#             response_text = "Error: Unable to generate detailed use case scenario."
#         return response_text
#     except Exception as e:
#         print(f"Error for story: {user_story} - {e}")
#         return "Error: Unable to generate use case scenario"

# Function to convert use case scenario to PlantUML sequence diagram script
def get_plantuml_sequence_diagram(user_story):
    """
    Uses OpenAI API to generate a PlantUML sequence diagram script based on the user story.
    """
    prompt = f"""
    You are an expert in software engineering and UML diagramming. Based on the following UML user story, generate a PlantUML script for a sequence diagram that visualizes the main flow of events. Only answer with the script.

    Use Case Scenario: "{user_story}"

    The PlantUML script should include all relevant actors and system components involved in the main flow.
    Ensure the script is syntactically correct and can be rendered without errors.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert in software engineering and UML diagramming."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0.3
        )
        plantuml_script = response['choices'][0]['message']['content'].strip()
        if "Error" in plantuml_script or not plantuml_script:
            plantuml_script = "@startuml\n' Error: Unable to generate sequence diagram\n@enduml"
        return plantuml_script
    except Exception as e:
        print(f"Error generating PlantUML script - {e}")
        return "@startuml\n' Error: Unable to generate sequence diagram\n@enduml"

# Directory to save PlantUML scripts and diagrams
plantuml_dir = "plantuml_sequences"
os.makedirs(plantuml_dir, exist_ok=True)

# Initialize PlantUML server (optional, if you have a local PlantUML server)
# If you don't have a local server, you can use the default PlantUML public server
plantuml_server = PlantUML(url='http://www.plantuml.com/plantuml/img/')  # Public server

# Generate use case scenarios and corresponding PlantUML sequence diagrams
# use_case_results = []
plantuml_scripts = []
sequence_diagrams = []

for idx, story in enumerate(user_stories, start=1):
    print(f"Processing User Story {idx}/{len(user_stories)}")
    # use_case = get_use_case_scenarios_in_uml(story)
    # use_case_results.append((story, use_case))

    # Generate PlantUML script
    plantuml_script = get_plantuml_sequence_diagram(story)
    plantuml_scripts.append((story, plantuml_script))

    # Save PlantUML script to a file
    sanitized_story = "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in story])[:50]
    puml_filename = f"{plantuml_dir}/sequence_{idx}_{sanitized_story}.puml"
    with open(puml_filename, 'w') as f:
        f.write(plantuml_script)

    # Optionally, generate and save the sequence diagram image
    try:
        # Using PlantUML public server to generate the image
        diagram_url = plantuml_server.get_url(plantuml_script)
        diagram_filename = f"{plantuml_dir}/sequence_{idx}_{sanitized_story}.png"
        # Download the image
        import requests
        img_data = requests.get(diagram_url).content
        with open(diagram_filename, 'wb') as img_file:
            img_file.write(img_data)
    except Exception as e:
        print(f"Error generating diagram for story {idx}: {e}")

    time.sleep(1)  # Add delay to avoid rate limits

# Save use case scenarios to a Word document
# doc = Document()
# doc.add_heading('UML Use Case Scenarios', level=1)

# for story, use_case in use_case_results:
#     doc.add_heading(f"User Story: {story}", level=2)
#     doc.add_paragraph(use_case)
#     doc.add_paragraph("\n")  # Add a blank line between entries

# doc.save("use_case_scenarios.docx")

# Save PlantUML scripts to a separate Word document (optional)
puml_doc = Document()
puml_doc.add_heading('PlantUML Sequence Diagrams', level=1)

for story, script in plantuml_scripts:
    puml_doc.add_heading(f"User Story: {story}", level=2)
    puml_doc.add_paragraph(script)
    puml_doc.add_paragraph("\n")  # Add a blank line between entries

puml_doc.save("plantuml_sequence_diagrams.docx")

# Notify the user
print("Word documents saved as 'use_case_scenarios.docx' and 'plantuml_sequence_diagrams.docx'.")
print(f"PlantUML scripts and sequence diagram images are saved in the '{plantuml_dir}' directory.")

# Download the Word files in Google Colab (if applicable)
try:
    from google.colab import files
    # files.download('use_case_scenarios.docx')
    files.download('plantuml_sequence_diagrams.docx')
    # Optionally, zip the PlantUML directory and download
    import shutil
    shutil.make_archive("plantuml_sequences", 'zip', plantuml_dir)
    files.download("plantuml_sequences.zip")
except ImportError:
    print("Google Colab is not being used, files saved locally.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>